In [1]:
import torch
from model import Unet_model
from Modules import STFT

In [2]:
fs = 16000
sec = 1
signal = torch.rand(1, 1, sec*fs) # 1 

# 25 ms window length

In [3]:
window_length = int(0.025 * fs)  # 25 ms window length
window_shift = int(0.01 * fs)  # 10 ms window shift
print(f'window_length: {window_length}')
print(f'window_shift: {window_shift}')
stft = STFT(n_fft = window_length, hop=window_shift)
spec = stft.stft(signal)
m = Unet_model(2)
spec.abs().shape

window_length: 400
window_shift: 160


torch.Size([1, 1, 200, 100])

In [4]:
print('Кол-во параметров:', sum(p.numel() for p in m.parameters() if p.requires_grad))

Кол-во параметров: 822287


## 1 sec inference

In [5]:
%%timeit
m(spec.abs())

17.2 ms ± 693 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 25 ms inference

In [6]:
single = torch.rand(1, 1, 200, 1)
single.shape

torch.Size([1, 1, 200, 1])

In [7]:
%%timeit
m(single.abs())

3.91 ms ± 189 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# 40 ms window length

In [8]:
window_length = int(0.04 * fs)  # 40 ms window length
window_shift = int(0.01 * fs)  # 10 ms window shift
print(f'window_length: {window_length}')
print(f'window_shift: {window_shift}')
stft = STFT(n_fft = window_length, hop=window_shift)
spec = stft.stft(signal)
m = Unet_model(4)
spec.abs().shape

window_length: 640
window_shift: 160


torch.Size([1, 1, 320, 100])

In [9]:
print('Кол-во параметров:', sum(p.numel() for p in m.parameters() if p.requires_grad))

Кол-во параметров: 2397199


Количество параметров выросло из за LSTM 

``LSTM: (input_size = 128*botlleneck, hidden_size = 128*botlleneck)``

(B, C, F, T) -> (B, C*F, T) -> (B, T, C*F)

F выросло -> input_size вырос

## 1 sec inference

In [10]:
%%timeit
m(spec.abs())

32.3 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 40 ms inference

In [11]:
single = torch.rand(1, 1, 320, 1)
single.shape

torch.Size([1, 1, 320, 1])

In [12]:
%%timeit
m(single.abs())

6.38 ms ± 328 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
